In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['EK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('electrokinetic', 11),
 ('enterokinase', 9),
 ('enkephalin', 7),
 ('ethanolamine kinase', 5),
 ('epidermal keratinocytes', 5),
 ('enamel knot', 4),
 ('ethylketocyclazocine', 4),
 ('electrophoretic karyotyping', 3),
 ('endothelial keratoplasty', 3),
 ('escherichia coli and klebsiella species', 3),
 ('k + equilibrium potential', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [57]:
grounding_map, names, pos_labels = ({'electrokinetic': 'ungrounded',
  'enkephalin': 'MESH:D004745',
  'enterokinase': 'HGNC:9490',
  'enteropeptidase': 'HGNC:9490',
  'epidermal keratinocytes': 'ungrounded',
  'ethanolamine kinase': 'HGNC:1937'},
 {'MESH:D004745': 'Enkephalins', 'HGNC:9490': 'TMPRSS15', 'HGNC:1937': 'CHKA'},
 ['HGNC:1937', 'HGNC:9490', 'MESH:D004745'])

In [58]:
excluded_longforms = []

In [59]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [60]:
additional_entities = {}

In [61]:
unambiguous_agent_texts = {}

In [62]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [46]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [47]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [48]:
intersection1

[]

In [49]:
intersection2

[]

In [50]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [51]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [63]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 01:29:59] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 01:30:00] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.73 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [64]:
classifier.stats

{'label_distribution': {'HGNC:9490': 8,
  'MESH:D004745': 7,
  'ungrounded': 15,
  'HGNC:1937': 4},
 'f1': {'mean': 0.73, 'std': 0.105093},
 'precision': {'mean': 0.75, 'std': 0.052705},
 'recall': {'mean': 0.733333, 'std': 0.161589},
 'ungrounded': {'f1': {'mean': 0.78619, 'std': 0.071909},
  'pr': {'mean': 0.866667, 'std': 0.163299},
  'rc': {'mean': 0.753333, 'std': 0.135565}},
 'HGNC:1937': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'HGNC:9490': {'f1': {'mean': 0.693333, 'std': 0.369023},
  'pr': {'mean': 0.7, 'std': 0.4},
  'rc': {'mean': 0.733333, 'std': 0.38873}},
 'MESH:D004745': {'f1': {'mean': 0.5, 'std': 0.447214},
  'pr': {'mean': 0.5, 'std': 0.447214},
  'rc': {'mean': 0.5, 'std': 0.447214}}}

In [65]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [66]:
disamb.dump(model_name, results_path)

In [67]:
print(disamb.info())

Disambiguation model for EK

Produces the disambiguations:
	CHKA*	HGNC:1937
	Enkephalins*	MESH:D004745
	TMPRSS15*	HGNC:9490

Class level metrics:
--------------------
Grounding  	Count	F1     
 Ungrounded	15	0.78619
   TMPRSS15*	 8	0.69333
Enkephalins*	 7	    0.5
       CHKA*	 4	    0.8

Weighted Metrics:
-----------------
	F1 score:	0.73
	Precision:	0.75
	Recall:		0.73333

* Positive labels
See Docstring for explanation



In [68]:
model_to_s3(disamb)